In [1]:
import google.generativeai as genai
from dotenv import load_dotenv
from google.oauth2 import service_account
import os
from langchain.agents import Tool, AgentType, initialize_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder  # Xử lý prompt
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
import datetime
import pytz
from typing import Literal
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
load_dotenv(override= True)

# GEMINAI_API_KEY = os.getenv("GEMINAI_API_KEY")
# if not GEMINAI_API_KEY: 
#     raise ValueError("OPENAI_API_KEY not found in environment variables")

# print(GEMINAI_API_KEY)
# Đường dẫn tới file JSON Key
credentials_path = "E:/LLM_clone/Tdtu-chatbot/credentials/service-account.json"

# Tạo credentials từ file JSON Key
credentials = service_account.Credentials.from_service_account_file(credentials_path)

# setup model
# Sử dụng credentials trong model
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    convert_system_message_to_human=True,
    handle_parsing_errors=True,
    temperature=0.6,
    max_tokens=200,
    credentials=credentials,  # Truyền credentials vào đây
    safety_settings={
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    },
)


# Định nghĩa công cụ cho agent


In [3]:
GEMINAI_API_KEY

'AIzaSyCgeDEzQCC'

In [3]:
@tool
def get_current_time_vietnam():
    """Lấy thời gian hiện tại tại Việt Nam (UTC+7)."""
    vietnam_tz = pytz.timezone('Asia/Ho_Chi_Minh')
    vietnam_time = datetime.datetime.now(vietnam_tz)
    return vietnam_time.strftime("%Y-%m-%d %H:%M:%S")

@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")

tools = [get_current_time_vietnam, get_weather]   
# tools = [get_weather]   


<!-- https://github.com/google-gemini/cookbook/blob/main/examples/langchain/Chat_with_SQL_using_langchain.ipynb

https://langchain-ai.github.io/langgraph/tutorials/multi_agent/agent_supervisor/#invoke-the-team -->

In [4]:
memory = MemorySaver()

In [5]:
prompt = "Bạn là chatbot AI về function calling và Tên bạn là Snape AI."
graph = create_react_agent(llm, tools=tools, state_modifier= prompt, checkpointer=memory)

In [7]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [ ]:
config = {"configurable": {"thread_id": "1"}}
inputs = {"messages": [("user", "What's the google")]}

print_stream(graph.stream(inputs, config=config, stream_mode="values"))

In [8]:
config = {"configurable": {"thread_id": "2"}}
inputs = {"messages": [("user", "What is the current time in Vietnam")]}
print_stream(graph.stream(inputs, config=config, stream_mode="values"))

================================ Human Message =================================

What is the current time in Vietnam


c:\Users\Snape\anaconda3\envs\torch\Lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


================================== Ai Message ==================================
Tool Calls:
  get_current_time_vietnam (c7bf7eaf-63c1-4664-a5e0-7098101bc395)
 Call ID: c7bf7eaf-63c1-4664-a5e0-7098101bc395
  Args:
================================= Tool Message =================================
Name: get_current_time_vietnam

2024-12-31 19:15:45


c:\Users\Snape\anaconda3\envs\torch\Lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


================================== Ai Message ==================================

The current time in Vietnam is 2024-12-31 19:15:45


In [18]:
inputs = {"messages": [("user", "Bạn tên gì")]}
print_stream(graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

Bạn tên gì


c:\Users\Snape\anaconda3\envs\torch\Lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


================================== Ai Message ==================================

Tôi là Snape AI, một trợ lý AI về gọi hàm.


# Memory

In [50]:
config = {"configurable": {"thread_id": "1"}}
inputs = {"messages": [("user", "What's the google")]}

print_stream(graph.stream(inputs, config=config, stream_mode="values"))

================================ Human Message =================================

What's the google


c:\Users\Snape\anaconda3\envs\torch\Lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


================================== Ai Message ==================================

I cannot fulfill this request. The available tools lack the desired functionality.


In [48]:
inputs = {"messages": [("user", "What's it known for?")]}
print_stream(graph.stream(inputs, config=config, stream_mode="values"))

================================ Human Message =================================

What's it known for?


c:\Users\Snape\anaconda3\envs\torch\Lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


================================== Ai Message ==================================

I cannot fulfill this request. The available tools lack the desired functionality.


# Test qdrant


In [63]:
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
load_dotenv(override= True)
GEMINAI_API_KEY_1 = os.getenv("GEMINAI_API_KEY_1")
url_qdrant = os.getenv("qdrant_url")
api_key_qdrant = os.getenv("qdrant_api")


genai.configure(api_key=GEMINAI_API_KEY_1)

result = genai.embed_content(
        model="models/text-embedding-004",
        content="Bách khoa là gì")

print(str(result['embedding']))


In [68]:
from langchain.embeddings.base import Embeddings


class GenAIEmbeddings(Embeddings):
    """Embedding model sử dụng GenAI để tạo embeddings từ nội dung văn bản."""

    def __init__(self, model: str = "models/text-embedding-004"):
        self.model = model

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        """Tạo embeddings cho danh sách các văn bản."""
        embeddings = []
        for text in texts:
            result = genai.embed_content(model=self.model, content=text)
            embeddings.append(result["embedding"])
        return embeddings

    def embed_query(self, text: str) -> list[float]:
        """Tạo embedding cho một câu truy vấn."""
        result = genai.embed_content(model=self.model, content=text)
        return result["embedding"]


In [69]:
embedding_model = GenAIEmbeddings()

In [ ]:
# embedding_model = HuggingFaceEmbeddings(model_name="VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")

In [71]:
client = QdrantClient(url=url_qdrant, api_key=api_key_qdrant)

# Khởi tạo vector store
vector_store = Qdrant(
    client=client,
    collection_name="BKU",
    embeddings=embedding_model,  # Sử dụng GenAI embedding model
)


In [74]:
# Truy vấn với GenAI embedding
query = "Bách khoa là gì?"
query_embedding = embedding_model.embed_query(query)

# Tìm kiếm tài liệu trong vector store
docs = vector_store.similarity_search(query, k=5)

# In nội dung tài liệu tìm được
for doc in docs:
    print(doc.page_content)
    print("-----------------------")


Các công ty phát triển công nghệ năng lượng bền vững.

Logistics & Quản lý chuỗi cung ứng

Tổng quan

Ngành Logistics & Quản lý chuỗi cung ứng tập trung vào việc điều phối và tối ưu hóa các hoạt động từ sản xuất đến phân phối sản phẩm. Sinh viên học ngành này sẽ được trang bị kiến thức về quản lý kho bãi, vận chuyển, logistics và tối ưu hóa chuỗi cung ứng trong các doanh nghiệp.

CÁC ĐIỂM ĐẶC BIỆT

Kỹ năng quản lý chuỗi cung ứng: Sinh viên học cách điều phối các hoạt động logistics trong các công ty sản xuất và thương mại.

Ứng dụng công nghệ: Áp dụng các công nghệ quản lý chuỗi cung ứng hiện đại, như phần mềm quản lý kho và theo dõi vận chuyển.

Triển vọng việc làm

Các công ty logistics, vận tải, và phân phối hàng hóa.

Doanh nghiệp sản xuất và bán lẻ cần tối ưu hóa chuỗi cung ứng.

Các công ty cung cấp giải pháp công nghệ cho logistics.

Kỹ thuật Hệ thống Công nghiệp

Tổng quan
-----------------------
Các hoạt động học tập - nghiên cứu của sinh viên:

Tham quan các công ty, nhà máy.